In [11]:
from sklearn import svm
from sklearn.model_selection import GridSearchCV
import os
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline

# Chargement des données
df_path = '../../data/processed/df_avec_categorie_part3_traduit.csv'
df = pd.read_csv(df_path)

# création de la colonne 'text' à partir de 'trad' ou 'truncated'
df['text'] = df.apply(lambda row: row['trad'] if not pd.isna(row['trad']) else row['truncated'], axis=1)

# Vectorisation TF-IDF
#vectorizer = TfidfVectorizer()
#X = vectorizer.fit_transform(df['text'])
X = df['text']
y = df['category_name']

# Split train/test stratifié pour conserver les proportions des classes
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Création du pipeline
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', svm.SVC(class_weight='balanced', random_state=42))
])

# Définition des paramètres à optimiser
param_grid = {
    # Paramètres TF-IDF
    'tfidf__min_df': [5, 10, 20],            # Ignorer les termes qui apparaissent dans moins de x documents
    'tfidf__max_df': [0.6, 0.8, 0.90],       # Ignorer les termes qui apparaissent dans plus de x% des documents
    'tfidf__ngram_range': [(1, 1), (1, 2)],  # Unigrammes ou uni+bigrammes
    'tfidf__max_features': [20000, 40000, None], # Limiter la taille du vocabulaire
    'tfidf__sublinear_tf': [True],           # Appliquer une mise à l'échelle log(1+tf) Presque toujours bénéfique pour de longs textes
    'tfidf__norm': ['l2'],                   # Normalisation des vecteurs (L2 est généralement le meilleur pour SVM)
    
    # Paramètres SVM 
    'clf__kernel': ['linear'],               # Linéaire est souvent meilleur pour du texte et plus rapide
    'clf__C': [0.5, 1, 5],                   # Plage plus ciblée pour équilibrer généralisation/surapprentissage
    'clf__class_weight': ['balanced'],       # Important pour les classes déséquilibrées
}

# Configuration de la recherche par grille
gridsearch = GridSearchCV(
    estimator=pipeline,
    param_grid=param_grid,
    cv=5,                    # Validation croisée à 5 plis
    scoring='f1_weighted',   # Métrique pertinente pour les classes déséquilibrées
    n_jobs=-1              # Utiliser tous les cœurs disponibles
)

gridsearch.fit(X,y)

print("les meilleurs paramètres sont", gridsearch.best_params_)
print("le meilleur score obtenu est ", gridsearch.best_score_)

#les meilleurs paramètres sont {'C': 10, 'class_weight': 'balanced', 'kernel': 'linear'}
#le meilleur score obtenu est  0.7450774234905061

#les meilleurs paramètres sont 
#{'clf__C': 1, 'clf__class_weight': 'balanced', 'clf__kernel': 'linear', 'tfidf__max_df': 0.6, 'tfidf__max_features': 40000, 'tfidf__min_df': 5, 'tfidf__ngram_range': (1, 2), 'tfidf__norm': 'l2', 'tfidf__sublinear_tf': True}
#le meilleur score obtenu est  0.7523168429429458


les meilleurs paramètres sont {'clf__C': 1, 'clf__class_weight': 'balanced', 'clf__kernel': 'linear', 'tfidf__max_df': 0.6, 'tfidf__max_features': 40000, 'tfidf__min_df': 5, 'tfidf__ngram_range': (1, 2), 'tfidf__norm': 'l2', 'tfidf__sublinear_tf': True}
le meilleur score obtenu est  0.7523168429429458


In [ ]:
from skopt import BayesSearchCV
parameters = {'kernel' : ['linear', 'rbf'], 'C' : (0.1, 10.0, 'log-uniform'), class_weight: 'balanced'}

svc_model = svm.SVC()
bayes_search = BayesSearchCV(svc_model, parameters,  n_iter = 20)
bayes_search.fit(X,y)

print("les meilleurs paramètres sont", bayes_search.best_params_)
print("le meilleur score obtenu est ", bayes_search.best_score_)

In [4]:
from sklearn import svm
from sklearn.model_selection import GridSearchCV
import os
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

# Chargement des données
df_path = '../../data/processed/df_avec_categorie_part3_traduit.csv'
df = pd.read_csv(df_path)

# création de la colonne 'text' à partir de 'trad' ou 'truncated'
df['text'] = df.apply(lambda row: row['trad'] if not pd.isna(row['trad']) else row['truncated'], axis=1)

# Vectorisation TF-IDF
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df['text'])
y = df['category_name']

vectorizer.get_feature_names_out()

array(['30', 'aa', 'aaa', ..., 'île', 'œil', 'œufs'],
      shape=(73642,), dtype=object)